# Exploração Driver Cassandra

Este notebook tem como objetivo explorar o funcionamento do driver do cassandra e seu _object mapper_ para a linguagem python.

In [1]:
import uuid
from datetime import datetime
from cassandra.cqlengine import connection, columns, ValidationError
from cassandra.cqlengine.models import Model
from cassandra.cqlengine.management import sync_table

## Criação e validação de modelos

In [67]:
class People(Model):
    """
        Model for people data table
    """
    __table_name__ = "people"
    id = columns.UUID(primary_key=True, default=uuid.uuid4)
    first_name = columns.Text()
    last_name = columns.Text(index=True)
    dt_insert = columns.Date(default=datetime.now())

In [3]:
connection.setup(hosts=["172.17.0.2"], default_keyspace="api_test", protocol_version=3)

In [4]:
sync_table(People)

/home/leobiscassi/.miniconda2/envs/python-playground/lib/python3.7/site-packages/cassandra/cqlengine/management.py:545: UserWarning: CQLENG_ALLOW_SCHEMA_MANAGEMENT environment variable is not set. Future versions of this package will require this variable to enable management functions.
  warnings.warn(msg)


In [72]:
farrell = People.create(first_name="Doug", last_name="Farrell")

In [73]:
brockman = People.create(first_name="Kent", last_name="Brockman")

In [74]:
easter = People.create(first_name="Bunny", last_name="Easter")

In [75]:
leo = People.create(last_name="Biscassi")

In [8]:
magno = People.create(first_name="Magno", last_name="Rodrigues")

ValidationError: Impossible to add son's of bitches

## Consultas

In [20]:
rows = People.objects.all()

In [10]:
rows

In [16]:
rows.first()

People(id=UUID('3e6a0076-50d2-4caa-a4e4-46c29a35efce'), first_name='Bunny', last_name='Easter', dt_insert=Date(17992))

In [18]:
for row in rows:
    print(row)
    print(dict(row))

People <id=3e6a0076-50d2-4caa-a4e4-46c29a35efce>
{'id': UUID('3e6a0076-50d2-4caa-a4e4-46c29a35efce'), 'first_name': 'Bunny', 'last_name': 'Easter', 'dt_insert': Date(17992)}
People <id=f83bdca1-4f00-4175-b8ef-b8dfe88882c1>
{'id': UUID('f83bdca1-4f00-4175-b8ef-b8dfe88882c1'), 'first_name': 'Doug', 'last_name': 'Farrell', 'dt_insert': Date(17992)}
People <id=2d297d1b-7efb-4214-9b1f-98a0741219b5>
{'id': UUID('2d297d1b-7efb-4214-9b1f-98a0741219b5'), 'first_name': 'Kent', 'last_name': 'Brockman', 'dt_insert': Date(17992)}


In [31]:
doug = rows.filter(last_name="Farrel")

In [32]:
doug.count()

0

In [33]:
doug = rows.filter(last_name="Farrell")

In [34]:
doug.count()

1

In [35]:
doug.get()

People(id=UUID('f83bdca1-4f00-4175-b8ef-b8dfe88882c1'), first_name='Doug', last_name='Farrell', dt_insert=Date(17992))

## Serializando tabelas cassandra para json com mashmallow

In [36]:
from marshmallow import Schema, fields, pprint

In [56]:
class PeopleSchema(Schema):
    id = fields.UUID(dump_only=True)
    first_name = fields.Str()
    last_name = fields.Str()
    dt_insert = fields.Str()

In [60]:
schema = PeopleSchema(many=True)

In [70]:
result = schema.dump(rows)

In [71]:
pprint(result, indent=2)

[ { 'dt_insert': '2019-04-06',
    'first_name': 'Bunny',
    'id': '3e6a0076-50d2-4caa-a4e4-46c29a35efce',
    'last_name': 'Easter'},
  { 'dt_insert': '2019-04-06',
    'first_name': 'Doug',
    'id': 'f83bdca1-4f00-4175-b8ef-b8dfe88882c1',
    'last_name': 'Farrell'},
  { 'dt_insert': '2019-04-06',
    'first_name': 'Kent',
    'id': '2d297d1b-7efb-4214-9b1f-98a0741219b5',
    'last_name': 'Brockman'}]
